In [1]:
"""
The code for testing the classifiers. Uses binary.csv and multiclass.csv

Loads feature lists and tuning ranges from pickles in CWD.

Change model types in models to tune on specific ones.

Writes output files.

Author: Wesley
"""

# Accelerates tuning of some classifiers
#from sklearnex import patch_sklearn
#patch_sklearn()

import pandas as pd
import numpy as np

from skopt import BayesSearchCV
from skopt.space import Categorical, Integer, Real

from sklearn.feature_selection import RFECV

from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler

from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
import xgboost as xgb
from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB

import pickle

from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler, LabelEncoder

from sklearn.model_selection import StratifiedKFold

from sklearn.metrics import (
    precision_score,
    recall_score,
    f1_score,
    confusion_matrix,
    accuracy_score
)

from time import time

from sklearn.metrics import classification_report

In [2]:
multiclass = pd.read_csv("multiclass_train.csv")
multiclass_test = pd.read_csv("multiclass_test.csv")

Preprocessing (make labels numeric)

In [3]:
# Encode attack labels to int and save as array to be used later.
le = LabelEncoder()
multiclass[" Label"] = le.fit_transform(multiclass[" Label"].values)
multiclass_test[" Label"] = le.transform(multiclass_test[" Label"].values)

multiclass_labels = []
print("\nMulticlass Label Encodings (in order of digits 0 -> n): ")
for i in range(0, len(list(set(list(multiclass[' Label']))))):
    multiclass_labels.append(le.inverse_transform([i])[0])

print(multiclass_labels)


Multiclass Label Encodings (in order of digits 0 -> n): 
['DNS', 'LDAP', 'MSSQL', 'NTP', 'NetBIOS', 'Portmap', 'SNMP', 'SSDP', 'Syn', 'TFTP', 'UDP', 'UDP-lag']


Load feature sets and search spaces and enumerate their contents.

In [4]:
feature_sets = pickle.load(open("feature_sets.pickle", 'rb'))
search_spaces = pickle.load(open("hyperparameter_search_spaces.pickle", 'rb'))

print(f"Available Tuning Ranges: {search_spaces.keys()}")

print("Feature Sets for Binary Dataset:")
for key, value in feature_sets["Binary"].items():
    if key == "RFE Sets":
        print(value.keys())

    elif key == "PCA":
        print(f"{key}, suggested variance threshold is {value}")
        
    else:
        print(key)

print("Feature Sets for Multiclass Dataset:")
for key, value in feature_sets["Multiclass"].items():
    if key == "RFE Sets":
        print(value.keys())

    elif key == "PCA":
        print(f"{key}, suggested variance threshold is {value}")

    else:
        print(key)

Available Tuning Ranges: dict_keys(['XGBoost', 'Bagging SVM', 'SVC (RBF)', 'SVC (Poly)', 'Logistic Regression', 'Random Forest', 'KNN', 'Linear SVC', 'Naive Bayes', 'Decision Tree'])
Feature Sets for Binary Dataset:
All
Correlation
Mutual Information
dict_keys(['Decision Tree', 'Random Forest', 'XGBoost', 'Linear SVC', 'Logistic Regression'])
PCA, suggested variance threshold is 0.95
Feature Sets for Multiclass Dataset:
All
Correlation
Mutual Information
dict_keys(['Decision Tree', 'Random Forest', 'XGBoost', 'Logistic Regression'])
PCA, suggested variance threshold is 0.95


In [5]:
"""
This is a helper method to place our performance results in a DataFrame for future analysis.
"""
def format_results_multiclass(y_test, predicted_values, fold_index, fitTime):
    # get scores
    accuracy = accuracy_score(y_test,predicted_values)
    recall_pos = recall_score(y_test, predicted_values, average='macro')
    precision_pos = precision_score(y_test,predicted_values, average='macro')
    f1 = f1_score(y_test,predicted_values, average='macro')

    cols = ["Fitting Time", "accuracy", "Precision", "Recall", "F1 Score"]
    results = [fitTime, accuracy, precision_pos, recall_pos, f1]

    outFrame = pd.DataFrame([results], columns=cols, index=[fold_index])

    return outFrame

This is for tuning on the multiclass set

In [6]:
models = {
    #"Decision Tree": DecisionTreeClassifier(random_state=42),
    #"Random Forest": RandomForestClassifier(random_state=42),
    #"XGBoost": xgb.XGBClassifier(random_state=42, num_class=11, objective='multi:softmax'),
    #"Linear SVC": make_pipeline(StandardScaler(), LinearSVC(random_state=42)),
    #"Logistic Regression": make_pipeline(StandardScaler(), LogisticRegression(random_state=42)),
    "KNN": KNeighborsClassifier(),
    #"Naive Bayes": GaussianNB(),
}

score_methods = ['accuracy']

feature_set = feature_sets["Multiclass"]

y = multiclass[" Label"].copy()
X = multiclass.drop([" Label"], axis=1)

y_test_f = multiclass[" Label"].copy()
X_test_f = multiclass.drop([" Label"], axis=1)

In [7]:
params = []

# This will hold all of our results.
runFrame = None

for name, model in models.items():
        for feature_key, feature_val in feature_set.items():

            # If we're on the RFE sets, check if we have one for this classifier. If not, skip it.
            if feature_key == "RFE Sets":
                if name in feature_val.keys():
                    feature_val = feature_val[name]
                else:
                    continue
            
            for score_method in score_methods:
                opt = BayesSearchCV(estimator=model,search_spaces=search_spaces[name],n_iter=50,scoring=score_method,cv=5,n_jobs=5)
                kf = StratifiedKFold(n_splits=5, random_state=42, shuffle=True)

                counter = 0

                # Used to hold data for a single run (performance metric)
                perfFrame = None

                for train_index, test_index in kf.split(X, y):

                    counter += 1

                    # PCA requires different logic to create X.
                    if feature_key != "PCA":
                        current_X = X.loc[:, feature_val]
                        X_train, X_test = current_X.iloc[train_index,:], current_X.iloc[test_index,:]
                        Y_train, Y_test = y.iloc[train_index], y.iloc[test_index]

                        current_X_f = X_test_f.loc[:, feature_val]
                        
                    else:
                        pca_trans = PCA(n_components=feature_val, random_state=42)
                        current_X = X.loc[:, X.columns]
                        X_train, X_test = current_X.iloc[train_index,:], current_X.iloc[test_index,:]
                        Y_train, Y_test = y.iloc[train_index], y.iloc[test_index]

                        # Apply PCA to training set and use it to transform test set.
                        X_train = pca_trans.fit_transform(X_train)
                        X_test = pca_trans.transform(X_test)
                        current_X_f = pca_trans.transform(X_test_f)

                        # Convert back to DataFrames
                        pca_cols = ["PC"+str(i) for i in list(range(1, len(X_train[0])+1))]
                        X_train = pd.DataFrame(data=X_train, columns=pca_cols)
                        X_test = pd.DataFrame(data=X_test, columns=pca_cols)
                        current_X_f = pd.DataFrame(data=current_X_f, columns=pca_cols)

                    startTime = time()

                    opt.fit(X_train,Y_train)

                    endTime = time()
                    fitTime = endTime - startTime

                    predicted_values = opt.predict(X_test)

                    # get metrics for this fold.
                    foldFrame = format_results_multiclass(Y_test, predicted_values, counter, fitTime)

                    # Add them to our lists of metric.
                    if perfFrame is None:
                        perfFrame = foldFrame
                    else:
                        perfFrame = pd.concat([perfFrame, foldFrame])

                    # Print a classification report on the testing results.
                    print("Validation Results: ")
                    print(classification_report(Y_test, predicted_values, target_names=multiclass_labels, digits=6))

                    print("Testing Results: ")
                    # Print a classification report on the testing results.
                    pred_test = opt.predict(current_X_f)
                    print(classification_report(y_test_f, pred_test, target_names=multiclass_labels, digits=6))

                    # Add tuple with the best params as well as the related model/config
                    params.append((f"Multiclass {name} {feature_key} {score_method} Fold {counter}", opt.best_params_))

                # Create a new line in the results table that averages all the folds
                perfFrame.loc["fold average"] = perfFrame.mean()

                # Mark the results table with the chosen classifier and the current performance metric.
                perfFrame['metric'] = [score_method for j in range(0,6)]
                perfFrame['Classifier'] = [name for j in range(0,6)]
                perfFrame['Feature Set'] = [feature_key for j in range(0,6)]
                perfFrame['Dataset'] = ["multiclass" for j in range(0,6)]
                print(f"{name} with {feature_key} and {score_method} completed.")

                # Add this run to the table with all runs.
                if runFrame is None:
                    runFrame = perfFrame
                else:
                    runFrame = pd.concat([runFrame, perfFrame])

# Write output file, best parameters, and best models to be used later.
runFrame.to_csv(f"multiclass_results_KNN_{time()}.csv")
pickle.dump(params, open(f"multiclass_params_KNN_{time()}.pickle", "wb"))

c:\Users\icarus\AppData\Local\Programs\Python\Python310\lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
c:\Users\icarus\AppData\Local\Programs\Python\Python310\lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
c:\Users\icarus\AppData\Local\Programs\Python\Python310\lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
c:\Users\icarus\AppData\Local\Programs\Python\Python310\lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "


Validation Results: 
              precision    recall  f1-score   support

         DNS   0.681671  0.550806  0.609291      2667
        LDAP   0.615492  0.661417  0.637629      2667
       MSSQL   0.907236  0.917104  0.912143      2666
         NTP   0.913855  0.954989  0.933969      2666
     NetBIOS   0.714777  0.468117  0.565730      2666
     Portmap   0.614432  0.855964  0.715361      2666
        SNMP   0.713879  0.752156  0.732518      2667
        SSDP   0.504340  0.435696  0.467512      2667
         Syn   0.909532  0.561680  0.694483      2667
        TFTP   0.986916  0.989876  0.988394      2667
         UDP   0.499216  0.596550  0.543560      2667
     UDP-lag   0.633275  0.810953  0.711184      2666

    accuracy                       0.712929     31999
   macro avg   0.724552  0.712942  0.709314     31999
weighted avg   0.724547  0.712929  0.709305     31999

Testing Results: 
              precision    recall  f1-score   support

         DNS   0.720956  0.583665  0.64

c:\Users\icarus\AppData\Local\Programs\Python\Python310\lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
c:\Users\icarus\AppData\Local\Programs\Python\Python310\lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
c:\Users\icarus\AppData\Local\Programs\Python\Python310\lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
c:\Users\icarus\AppData\Local\Programs\Python\Python310\lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
c:\Users\icarus\AppData\Local\Programs\Python\Python310\lib\site-packages\skopt\optimize

Validation Results: 
              precision    recall  f1-score   support

         DNS   0.578194  0.636534  0.605963      2666
        LDAP   0.610096  0.475816  0.534653      2667
       MSSQL   0.893452  0.905851  0.899609      2666
         NTP   0.917477  0.954989  0.935857      2666
     NetBIOS   0.715385  0.453488  0.555096      2666
     Portmap   0.605795  0.854518  0.708975      2667
        SNMP   0.726535  0.772028  0.748591      2667
        SSDP   0.509398  0.508061  0.508729      2667
         Syn   0.861780  0.577428  0.691513      2667
        TFTP   0.983551  0.986502  0.985024      2667
         UDP   0.520246  0.539558  0.529726      2667
     UDP-lag   0.626050  0.782446  0.695565      2666

    accuracy                       0.703928     31999
   macro avg   0.712330  0.703935  0.699942     31999
weighted avg   0.712325  0.703928  0.699936     31999

Testing Results: 
              precision    recall  f1-score   support

         DNS   0.613823  0.707418  0.65

c:\Users\icarus\AppData\Local\Programs\Python\Python310\lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
c:\Users\icarus\AppData\Local\Programs\Python\Python310\lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
c:\Users\icarus\AppData\Local\Programs\Python\Python310\lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
c:\Users\icarus\AppData\Local\Programs\Python\Python310\lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
c:\Users\icarus\AppData\Local\Programs\Python\Python310\lib\site-packages\skopt\optimize

...

Validation Results: 
              precision    recall  f1-score   support

         DNS   0.715859  0.487622  0.580098      2666
        LDAP   0.603033  0.685789  0.641754      2667
       MSSQL   0.894398  0.886014  0.890186      2667
         NTP   0.884397  0.878095  0.881235      2666
     NetBIOS   0.714110  0.434571  0.540326      2667
     Portmap   0.603154  0.874766  0.714002      2667
        SNMP   0.708224  0.807577  0.754644      2666
        SSDP   0.522157  0.570143  0.545096      2666
         Syn   0.866546  0.538058  0.663891      2667
        TFTP   0.975185  0.987252  0.981181      2667
         UDP   0.545235  0.508627  0.526295      2666
     UDP-lag   0.622763  0.835021  0.713439      2667

    accuracy                       0.707803     31999
   macro avg   0.721255  0.707794  0.702679     31999
weighted avg   0.721262  0.707803  0.702686     31999

Testing Results: 
              precision    recall  f1-score   support

         DNS   0.785753  0.538588  0.63

c:\Users\icarus\AppData\Local\Programs\Python\Python310\lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
c:\Users\icarus\AppData\Local\Programs\Python\Python310\lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
c:\Users\icarus\AppData\Local\Programs\Python\Python310\lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
c:\Users\icarus\AppData\Local\Programs\Python\Python310\lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
c:\Users\icarus\AppData\Local\Programs\Python\Python310\lib\site-packages\skopt\optimize

Validation Results: 
              precision    recall  f1-score   support

         DNS   0.596549  0.609299  0.602857      2667
        LDAP   0.626016  0.490998  0.550347      2666
       MSSQL   0.905718  0.896888  0.901281      2667
         NTP   0.891800  0.880765  0.886248      2667
     NetBIOS   0.690617  0.482940  0.568402      2667
     Portmap   0.607163  0.820022  0.697719      2667
        SNMP   0.703211  0.813203  0.754218      2666
        SSDP   0.538089  0.582896  0.559597      2666
         Syn   0.824875  0.554764  0.663377      2666
        TFTP   0.976279  0.987997  0.982103      2666
         UDP   0.557240  0.524006  0.540112      2666
     UDP-lag   0.622120  0.809899  0.703698      2667

    accuracy                       0.704482     31998
   macro avg   0.711640  0.704473  0.700830     31998
weighted avg   0.711641  0.704482  0.700835     31998

Testing Results: 
              precision    recall  f1-score   support

         DNS   0.649551  0.645166  0.64

c:\Users\icarus\AppData\Local\Programs\Python\Python310\lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
c:\Users\icarus\AppData\Local\Programs\Python\Python310\lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
c:\Users\icarus\AppData\Local\Programs\Python\Python310\lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
c:\Users\icarus\AppData\Local\Programs\Python\Python310\lib\site-packages\skopt\optimizer\optimizer.py:449: UserWarning: The objective has been evaluated at this point before.
  warnings.warn("The objective has been evaluated "
c:\Users\icarus\AppData\Local\Programs\Python\Python310\lib\site-packages\skopt\optimize

Validation Results: 
              precision    recall  f1-score   support

         DNS   0.736750  0.474316  0.577099      2667
        LDAP   0.600463  0.681545  0.638440      2666
       MSSQL   0.886834  0.899138  0.892944      2667
         NTP   0.889566  0.875891  0.882675      2667
     NetBIOS   0.744575  0.450319  0.561215      2667
     Portmap   0.611430  0.878845  0.721145      2666
        SNMP   0.703560  0.815454  0.755386      2666
        SSDP   0.521165  0.503376  0.512116      2666
         Syn   0.860588  0.548762  0.670179      2666
        TFTP   0.972603  0.985371  0.978945      2666
         UDP   0.520263  0.563180  0.540871      2667
     UDP-lag   0.625744  0.827522  0.712625      2667

    accuracy                       0.708638     31998
   macro avg   0.722795  0.708643  0.703637     31998
weighted avg   0.722797  0.708638  0.703635     31998

Testing Results: 
              precision    recall  f1-score   support

         DNS   0.799200  0.524488  0.63

In [9]:
best = pickle.load(open("multiclass_params_KNN_1670628152.649287.pickle", 'rb'))

In [10]:
for i in range(0, 5):
    print(best[i])

('Multiclass KNN All accuracy Fold 1', OrderedDict([('algorithm', 'kd_tree'), ('n_neighbors', 29), ('weights', 'distance')]))
('Multiclass KNN All accuracy Fold 2', OrderedDict([('algorithm', 'kd_tree'), ('n_neighbors', 24), ('weights', 'distance')]))
('Multiclass KNN All accuracy Fold 3', OrderedDict([('algorithm', 'kd_tree'), ('n_neighbors', 47), ('weights', 'distance')]))
('Multiclass KNN All accuracy Fold 4', OrderedDict([('algorithm', 'kd_tree'), ('n_neighbors', 50), ('weights', 'distance')]))
('Multiclass KNN All accuracy Fold 5', OrderedDict([('algorithm', 'ball_tree'), ('n_neighbors', 50), ('weights', 'distance')]))


In [11]:
clf = KNeighborsClassifier(algorithm = 'kd_tree', n_neighbors=47, weights = 'distance')
clf.fit(X, y)
pred_test = clf.predict(X_test_f)
print(classification_report(y_test_f, pred_test, target_names=multiclass_labels, digits=6))

              precision    recall  f1-score   support

         DNS   0.650171  0.684842  0.667056     13333
        LDAP   0.666569  0.511288  0.578693     13333
       MSSQL   0.966134  0.964974  0.965553     13333
         NTP   0.997082  0.999700  0.998389     13332
     NetBIOS   0.740944  0.477087  0.580436     13333
     Portmap   0.620312  0.867172  0.723258     13333
        SNMP   0.733127  0.854710  0.789264     13332
        SSDP   0.801893  0.756301  0.778430     13332
         Syn   0.958987  0.596265  0.735328     13333
        TFTP   0.997816  0.993550  0.995678     13333
         UDP   0.773242  0.824046  0.797836     13333
     UDP-lag   0.679556  0.913448  0.779331     13333

    accuracy                       0.786947    159993
   macro avg   0.798819  0.786948  0.782438    159993
weighted avg   0.798818  0.786947  0.782436    159993

